<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-2/partB/partB_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Before executing the code, allocate GPU in Colab from: Edit -> Notebook Settings -> Hardware accelerator -> GPU

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
import random
import os
from google.colab import drive
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
import tensorflow.keras as K



#Model
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import ResNet50
from keras.applications import InceptionResNetV2
from keras.applications import Xception



In [ ]:
#To access content from google drive
drive.mount('/content/gdrive')

In [ ]:
#Todo- first rename the file in drive
#Use this link: https://drive.google.com/drive/folders/1yu5DuYlM_cvV5qwIKKnl3sHRG_G41aLB?usp=sharing
#To make copy of pickled data and save it folder name 'Raw_data350'

with open('/content/gdrive/My Drive/Raw_data350/xtrain','rb') as xT:
  xTrain = pickle.load(xT)

with open('/content/gdrive/My Drive/Raw_data350/ytrain','rb') as yT:
  yTrain = pickle.load(yT)

with open('/content/gdrive/My Drive/Raw_data350/xtest','rb') as xV:
  xTest = pickle.load(xV)

with open('/content/gdrive/My Drive/Raw_data350/ytest','rb') as yV:
  yTest = pickle.load(yV)

In [ ]:
# resizing xTrain and xTest
for i in range(len(xTrain)):
  xTrain[i]=xTrain[i].resize((224,224))

for i in range(len(xTest)):
  xTest[i]=xTest[i].resize((224,224))

In [ ]:
#Converting xTrain and xTest to numpy array 
for i in range(len(xTrain)):

  xTrain[i]=np.array(xTrain[i])
  
  if(xTrain[i].ndim==2):
    del xTrain[i]
    del yTrain[i]

xTrain=np.array(xTrain)

for i in range(len(xTest)):

  xTest[i]=np.array(xTest[i])
  
  if(xTest[i].ndim==2):
    del xTest[i]
    del yTest[i]

xTest=np.array(xTest)


# type(xTrain)

In [ ]:
#Converting yTrain and yTest to integer classes
from sklearn import preprocessing # change the string labels to integers 

le = preprocessing.LabelEncoder()
yTest=le.fit_transform(yTest)
yTrain=le.fit_transform(yTrain)

In [ ]:
#Converting to categorical data i.e. one hot vector
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

yTrain_cat=[]
for i in range(len(yTrain)):
  cat_d=to_categorical(int(yTrain[i]),10)
  yTrain_cat.append(cat_d)

yTest_cat=[]
for i in range(len(yTest)):
  cat_d=to_categorical(int(yTest[i]),10)
  yTest_cat.append(cat_d)

In [ ]:
#Conveting yTrain_cat and yTest_cat to numpy array
yTrain_cat=np.array(yTrain_cat)
yTest_cat=np.array(yTest_cat)

In [ ]:
yTest_cat.shape

In [ ]:
xTrain[1].shape

In [ ]:
#Function to alot model based input shape and number of output neuron
def create_model(input_shape, n_out, hiddenLayers, model_type='InceptionResNetV2'):

    if model_type=='VGG16':
      pretrain_model= VGG16(
          weights='imagenet', 
          include_top=False, 
          input_shape=input_shape)
      
    elif model_type=='VGG19':
      pretrain_model = VGG19(
          include_top=False, 
          weights='imagenet', 
          input_shape=input_shape) 
      

    elif model_type=='ResNet50':
      pretrain_model = ResNet50(
          include_top=False, 
          weights="imagenet", 
          input_shape=input_shape)


    elif model_type=='InceptionResNetV2':
      pretrain_model = InceptionResNetV2(
          include_top=False, 
          weights='imagenet', 
          input_shape=input_shape)   


    elif model_type=='Xception':
      pretrain_model = Xception(
          include_top=False, 
          weights='imagenet', 
          input_shape=input_shape) 

    x=pretrain_model.output
    x = Flatten()(x)

    if hiddenLayers=='Yes':
      x = Dense(512, activation='relu')(x)
      x = Dropout(0.5)(x)
      x = Dense(128, activation='relu')(x)
      x = Dropout(0.5)(x)


    x = Dense(n_out, activation='softmax')(x)
    model=Model(pretrain_model.input,x)
    
    return model

In [ ]:
"""model structure decided here"""


def model_layer_freeze(input_shape, n_out, hiddenLayers, trainLayers, model_type='InceptionResNetV2'):

    model = create_model(input_shape, n_out, hiddenLayers, model_type)

    if trainLayers=='Yes':
      return model

    
    if model_type=='VGG16':
      
     
      for layer in model.layers[:15]:
        layer.trainable = False

      
    elif model_type=='VGG19':
      
      for layer in model.layers[:17]:
        layer.trainable = False

    elif model_type=='ResNet50':
      
      for layer in model.layers[:143]:
        layer.trainable = False

    elif model_type=='InceptionResNetV2':   

      model.layers[2].trainable = False    

    elif model_type=='Xception':
      
      for layer in model.layers[:116]:
        layer.trainable = False

    return model
    


In [ ]:
#Wandb Part

In [ ]:
!pip install wandb

In [ ]:
import wandb
!wandb login

In [ ]:
sweep_config_temp={
  "name": "DL Assign-2-partB-ed3",
  "method": "random",
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  "parameters": {
        
        "model":{
            "values":['VGG16','VGG19', 'ResNet50', 'InceptionResNetV2', 'Xception']
        },
        
        "Hidden_Layers":{
            "values":['Yes', 'No']
        },
        "Layers_Trainable":{
            "values":['Yes', 'No']
        },

        "Learning_Rate":{
            "values":[1e-3, 2e-5]
        },

    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config_temp, entity="cs20m014", project="DL Assign-2-partB-ed3")

In [ ]:
hyperparameter_defaults = dict(
    model='InceptionResNetV2',
    Hidden_Layers='Yes',
    Layers_trainable='No',
    Learning_Rate=2e-5,
    )

In [ ]:
train_datagen = ImageDataGenerator(zoom_range=0.2, rotation_range=30, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2)


In [ ]:
train_generator = train_datagen.flow(xTrain,yTrain_cat,batch_size=32)


In [ ]:
train_datagen.fit(xTrain)

In [ ]:
yTest_cat.shape

In [ ]:
def testModel():

  keras.backend.clear_session()

  wandb.init(config=hyperparameter_defaults)
  config = wandb.config

  model_type = config.model
  learningRate=config.Learning_Rate
  hLayers=config.Hidden_Layers
  trainL=config.Layers_Trainable

  model = model_layer_freeze(
    input_shape=(224,224,3), 
    n_out=10,
    hiddenLayers=hLayers,
    trainLayers=trainL,
    model_type = model_type)
  
  model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.Adam(lr=learningRate), 
                  metrics=['accuracy'])

  history = model.fit(train_generator, batch_size=32, epochs=5, verbose=1,
                        validation_data=(xTest, yTest_cat)
                      )
  
  
  val_accuracy = max(history.history['val_accuracy'])
  train_accuracy = max(history.history['accuracy'])

  wandb.log({
      "Train Acc": train_accuracy,
        "Valid Acc": val_accuracy
        })

In [ ]:
yTrain_cat.shape

In [ ]:
wandb.agent(sweep_id, testModel)